In [1]:
import json
import os
from tqdm import tqdm

data_dir = '../data/'
dirs = ['BALANCE_SHEET', 'CASH_FLOW', 'INCOME_STATEMENT', 'EARNINGS']
TICKER_LIST = '../snp500sym.csv'
write_dir = '../data/QuarterlyData'

if not os.path.exists(write_dir):
    os.makedirs(write_dir)
    print(f'Created directory {write_dir}')

Created directory ../data/QuarterlyData


In [2]:
# Create List of keys
keys = ['ticker']
for d in dirs:
    f = open(os.path.join(data_dir, d, 'MMM.json'), 'r')
    data = json.load(f)
    try:
        quarter_data = data['quarterlyEarnings'][0] if d == 'EARNINGS' else data['quarterlyReports'][0]
    except:
        continue
    for key in quarter_data.keys():
        if key != 'fiscalDateEnding' and key not in keys:
            keys.append(key)
print(keys)

['ticker', 'reportedCurrency', 'totalAssets', 'totalCurrentAssets', 'cashAndCashEquivalentsAtCarryingValue', 'cashAndShortTermInvestments', 'inventory', 'currentNetReceivables', 'totalNonCurrentAssets', 'propertyPlantEquipment', 'accumulatedDepreciationAmortizationPPE', 'intangibleAssets', 'intangibleAssetsExcludingGoodwill', 'goodwill', 'investments', 'longTermInvestments', 'shortTermInvestments', 'otherCurrentAssets', 'otherNonCurrrentAssets', 'totalLiabilities', 'totalCurrentLiabilities', 'currentAccountsPayable', 'deferredRevenue', 'currentDebt', 'shortTermDebt', 'totalNonCurrentLiabilities', 'capitalLeaseObligations', 'longTermDebt', 'currentLongTermDebt', 'longTermDebtNoncurrent', 'shortLongTermDebtTotal', 'otherCurrentLiabilities', 'otherNonCurrentLiabilities', 'totalShareholderEquity', 'treasuryStock', 'retainedEarnings', 'commonStock', 'commonStockSharesOutstanding', 'operatingCashflow', 'paymentsForOperatingActivities', 'proceedsFromOperatingActivities', 'changeInOperatingLia

In [3]:
def get_filename(date):
    tokens = date.split('-')
    year = int(tokens[0])
    month = int(tokens[1])
    if month % 3 == 0:
        quarter = month // 3
    elif (month + 1) % 3 == 0:
            quarter = (month + 1) // 3
    elif (month - 1) % 3 == 0:
        if month == 1:
            quarter = 4
            year = year - 1
        else:
            quarter = (month - 1) // 3
    filename = f'{year}_Q{quarter}'
    return filename

In [4]:
f = open(TICKER_LIST, 'r')
for line in tqdm(f.readlines()):
    ticker = line.strip()
    for d in dirs:
        if d == 'EARNINGS':
            continue
        f = os.path.join(data_dir, d, ticker + '.json')
        data = json.load(open(f, 'r'))
        try:
            quarter_data = data['quarterlyReports']
        except:
            continue
        dates = [d['fiscalDateEnding'] for d in quarter_data]
        for date in dates:
            filename = os.path.join(write_dir, get_filename(date)+'.csv')
            if os.path.exists(filename):
                continue
            else:
                fw = open(filename, 'w')
                fw.write(','.join(keys) + '\n')
                fw.close()

100%|██████████| 505/505 [00:01<00:00, 338.79it/s]


In [5]:
f = open(TICKER_LIST, 'r')
for line in tqdm(f.readlines()):
    ticker = line.strip()
    data_dict = {}
    for d in dirs:
        f = os.path.join(data_dir, d, ticker + '.json')
        data = json.load(open(f, 'r'))
        try:
            quarter_data = data['quarterlyEarnings'] if d == 'EARNINGS' else data['quarterlyReports']
        except:
            continue
        for d in quarter_data:
            q_date = get_filename(d['fiscalDateEnding'])
            if q_date not in data_dict:
                data_dict[q_date] = d
            else:
                data_dict[q_date].update(d)
    for q_date, q_data in data_dict.items():
        filename = os.path.join(write_dir, q_date+'.csv')
        if os.path.exists(filename):
            fw = open(filename, 'a')
            l = []
            for key in keys:
                if key == 'ticker':
                    l.append(ticker)
                elif key in q_data:
                    l.append(q_data[key])
                else:
                    l.append('')
            fw.write(','.join(l) + '\n')
            fw.close()

100%|██████████| 505/505 [00:03<00:00, 138.85it/s]
